In [12]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-23 14:39:31--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-23 14:39:32 (11.9 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [16]:
!wget https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json

--2024-06-23 14:43:59--  https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json
Resolving github.com (github.com)... 20.26.156.215
Connecting to github.com (github.com)|20.26.156.215|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexeygrigorev/llm-rag-workshop/main/notebooks/documents.json [following]
--2024-06-23 14:43:59--  https://raw.githubusercontent.com/alexeygrigorev/llm-rag-workshop/main/notebooks/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.01s   

2024-06-23 14:44:00 (54.0 MB/s) - ‘documents.json’ saved [6583

In [5]:
import minsearch

In [6]:
import json

In [7]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [8]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [9]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [10]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [11]:
q = 'the course has already started, can I still enroll?'

In [12]:
index.fit(documents)

In [18]:
from openai import OpenAI

In [19]:
client = OpenAI()

In [20]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content":q}]
)

In [21]:
response.choices[0].message.content

"The possibility of enrolling in a course that has already started typically depends on the institution and the specific policies of the course or program. Here are a few steps you can take to find out:\n\n1. **Check the Course Website**: Look for any information on late registration or enrollment policies.\n\n2. **Contact the Instructor**: Reach out to the course instructor or professor. Often, they have the discretion to allow late enrollments, especially if the course is still in its early stages.\n\n3. **Speak with the Registrar's Office**: The registrar or admissions office at your institution can provide definitive information on whether you can still enroll and what the process would be.\n\n4. **Review the Academic Calendar**: Some institutions have specific deadlines for adding or dropping courses. Check to see if you're still within that window.\n\n5. **Consider the Course Load**: Keep in mind that catching up on missed material might be challenging, so assess whether you have

In [26]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [27]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [28]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [29]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [30]:
rag(query)

'To run Kafka in your project, follow these steps based on the context provided:\n\n1. **Java Kafka**:\n   - If you need to run a Java Kafka producer, consumer, kstreams, etc., navigate to your project directory and execute:\n     ```\n     java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n     ```\n\n2. **Python Kafka**:\n   - If you encounter the "Module \'kafka\' not found" error while running `producer.py`, you should create a virtual environment and install the required Python packages:\n     ```\n     python -m venv env\n     source env/bin/activate\n     pip install -r ../requirements.txt\n     ```\n     - Activate the virtual environment before running your Python files:\n       ```\n       source env/bin/activate\n       ```\n     - Deactivate it when done:\n       ```\n       deactivate\n       ```\n     - For Windows, the activation command is slightly different:\n       ```\n       env\\Scripts\\activate\n       ```'

In [31]:
rag('the course has already started, can I still enroll?')

'Yes, you can still enroll in the course after it has started. You will be eligible to submit the homeworks, but keep in mind there will be deadlines for turning in the final projects. So, try not to leave everything for the last minute.'

In [32]:
from elasticsearch import Elasticsearch

In [33]:
es_client = Elasticsearch('http://localhost:9200') 

In [34]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [35]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████████████████████████| 948/948 [00:19<00:00, 49.72it/s]


In [37]:
query = 'I just disovered the course. Can I still join it?'

In [38]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [39]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [41]:
rag(query)

"Yes, you can still join the course even if you just discovered it. You don't need to register to submit the homeworks. However, keep in mind that there will be deadlines for turning in the final projects, so don't wait until the last minute to catch up."